## ALR Client Side

This notebook is a copy similar to ALR_Client_Side found in:
https://github.com/rfernand387/ALR_Earth_Engine/blob/master/ALR_Client_Side.ipynb

The input image is assumed to be in the format generated by SL2P10_control.ipynb:
https://github.com/kateharvey/Sentinel2_ALR/blob/main/shared/SL2P10_control.ipynb

In [ ]:
import ee
import scipy
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn as skl ; from sklearn import preprocessing ; from sklearn import linear_model
import tensorflow as tf
import json
from collections import OrderedDict
import time
import math
import csv
import os
import geemap
import image_bands as ib
import feature_collections as fc

# import custom module
import ALR_functions_richard as alr


The jupyter notebook below has code blocks required to apply a  neural network that is uploaded to Earth Engine to an image.  Make sure the image has the correct bands that the network used when it was calibrated (including the same scaling).   I found the solution slow so maybe try it on a small image subset first.

In [ ]:
ee.Initialize()

Currently the trimmed data is processed then in a neural network created using tensorflow to find nonlinear relationships between the predictor and the response. Earth Engine does not have this functionality (for free) to generate neural network based models.

Here we also see how the server side in the Earth Engine API is completely separate from the client side on the local machine. We need
to export our trimmed data as a CSV to a google drive which is synced into the "gdrive" folder in our local machine using the 
Backup and Sync software or using google-drive-ocamlfuse on Linux

For a more robust way to check if the data has been exported properly, we can use a wait loop to check on our local machine until the exported data file exists in the synced "gdrive" folder

From here on out, all of the processing is done using your local hardware and packages, so it may be helpful to use a powerful machine.

In [ ]:
def parse_layer(feature):
    feature = ee.Feature(feature)
    prev_layer_size = feature.getNumber("prev_layer_size")
    num_nodes = feature.getNumber("num_nodes")
    node_size = prev_layer_size.subtract(1)
    activation = feature.getString("activation")
    
    node_collection = ee.ImageCollection(ee.List.sequence(1, num_nodes)\
                        .map(lambda node: ee.ImageCollection(ee.List.sequence(ee.Number(node).toInt(), ee.Number(node)\
                                    .toInt().add(node_size.multiply(num_nodes)), num_nodes)\
                        .map(lambda index: ee.Image(feature.getNumber(ee.Number(index).toInt())))).toBands()\
                        .set({"bias": feature.getNumber(ee.Number(node).toInt().add(prev_layer_size.multiply(num_nodes)))})))
    
    return ee.List([node_collection, activation])

In [ ]:
# parse neural net from gee asset
nnet = ee.FeatureCollection("users/hlahiouel/nnet")
nnet_inputs = nnet.filter(ee.Filter.eq("layer_num", 0)).first()
num_inputs = nnet_inputs.getNumber("num_nodes")
nnet = nnet.filterBounds(ee.Geometry.Point([0,0]))
layer_list = nnet.sort("layer_num").toList(nnet.size())
neural_net = layer_list.map(parse_layer)

In [ ]:
COLLECTION_OPTIONS = {
    # Sentinel 2 using 20 m bands:
    'COPERNICUS/S2_SR': {
      "name": 'COPERNICUS/S2_SR',
      "description": 'Sentinel 2A',
      "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      "Watercover": 'WATER_PERCENTAGE',
      "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(fc.s2_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(fc.s2_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(fc.s2_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(fc.s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(fc.s2_createImageCollection_partition()),
      "legend": ee.FeatureCollection(fc.s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
    'LANDSAT/LC08/C01/T1_SR': {
      "name": 'LANDSAT/LC08/C01/T1_SR',
      "description": 'LANDSAT 8',
      "Cloudcover": 'CLOUD_COVER_LAND',
      "Watercover": 'CLOUD_COVER',
      "sza": 'SOLAR_ZENITH_ANGLE',
      "vza": 'SOLAR_ZENITH_ANGLE',
      "saa": 'SOLAR_AZIMUTH_ANGLE', 
      "vaa": 'SOLAR_AZIMUTH_ANGLE',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(fc.l8_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(fc.l8_createFeatureCollection_errors()),
      "sl2pDomain": ee.FeatureCollection(fc.l8_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(fc.l8_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(fc.l8_createImageCollection_partition()),
      "legend": ee.FeatureCollection(fc.l8_createFeatureCollection_legend()),
      "numVariables": 7
    }
}

VIS_OPTIONS = {
    'fAPAR': {
        "COPERNICUS/S2_SR": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        }
    },
    'fCOVER': {
        "COPERNICUS/S2_SR": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
        }
    },
    'LAI': {
        "COPERNICUS/S2_SR": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        }
        }
}

In [ ]:
# parse the networks
colName = 'COPERNICUS/S2_SR'
colOptions = COLLECTION_OPTIONS[colName]

In [ ]:
inputImage = ee.Image('COPERNICUS/S2_SR/20200811T164849_20200811T165525_T16UEA')

mapBounds = inputImage.geometry()

input_collection = ee.ImageCollection(inputImage) \
                         .map(lambda image: ib.addDate(image)) \
                         .map(lambda image: image.clip(mapBounds)) \
                         .map(lambda image: ib.s2MaskClear(image)) \
                         .map(lambda image: ib.s2MaskLand(image)) \
                         .map(lambda image: ib.addS2Geometry(colOptions, image))

In [ ]:
inputImage = input_collection.first()

input_nvdi = (inputImage.select('B8').subtract(inputImage.select('B4'))).divide(inputImage.select('B8').add(inputImage.select('B4')).add(ee.Image.constant(0.01))).rename('NVDI')

inputImage = inputImage.addBands(input_nvdi)

In [ ]:
LAI_inputs = ['B3','B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12','cosVZA', 'cosSZA', 'cosRAA', 'NVDI']

select_features = ['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'cosVZA', 'cosSZA', 'cosRAA', 'NVDI']

inputImage = inputImage.select(LAI_inputs).rename(select_features)

In [ ]:
print(inputImage.bandNames().getInfo())

In [ ]:
# define input image for network and apply network
# make sure input bands match inputs when network was trained, including any scaling
nnet_inputs = inputImage.select(select_features)
#print(inputImage.bandNames().getInfo())
#prediction_data = ee.Image(neural_net.iterate(alr.apply_nnet, nnet_inputs)).rename("NNET")

In [ ]:
# display result using geemap
Map = geemap.Map(center=[40,-100], zoom=4)
Map
Map.addLayer(prediction_data)

In [ ]:
Map.addLayer(nnet_inputs)